In [2]:
from keras.models import clone_model, load_model
from PIL import Image
import numpy as np
import os
from skimage.color import rgb2lab, lab2rgb

import cv2 as cv
from keras.losses import MeanSquaredError
from keras.utils import CustomObjectScope
from keras.metrics import mean_squared_error
from datetime import datetime

In [3]:
mse = mean_squared_error



with CustomObjectScope({'mse': mse}):
    model = load_model("Models\\humans_portrait.h5")
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

with CustomObjectScope({'mse': mse}):
    sharp = load_model("SR_test3.h5")
    sharp.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [4]:
def predict_model(image): 
    image1 = (np.array(image).astype("float32")/255)
    image1= image1.reshape(-1, image.shape[0], image.shape[1], 3)
    image2 = sharp.predict(image1)


    image2 = image2.reshape(image2.shape[1], image2.shape[2], image2.shape[3])
    cur = np.zeros((image2.shape[0], image2.shape[1], 3), dtype=np.float32)
    cur[:,:,:] = image2[:, :, :]
    return cur


In [4]:

def color_me2(image1, model):
    original_shape = image1.shape[:2]
    #image1 = np.array(image1, dtype=float)
    X = rgb2lab(1.0/255*image1)[:, :, 0]
    
    X = X.reshape(1, original_shape[0], original_shape[1], 1)

    output = model.predict(X)
    output = output * 128
    
    # output_resized = resize(output[0], (original_shape[0], original_shape[1], 2))

    cur = np.zeros((original_shape[0], original_shape[1], 3))
    cur[:, :, 0] = X[0, :, :, 0]
    # Burada output yerine output_resized kullanılabilir.
    cur[:, :, 1:] = output[0][:original_shape[0], :original_shape[1], :]

    # Y = (1.0/255*cur)[:,:,1:]
    # Y /= 128
    
    # Y = Y.reshape(1, original_shape[0], original_shape[1], 2)   
    
    # print(model.evaluate(X, Y, batch_size=1))
    
    return lab2rgb(cur)

In [8]:


def colorize_video(input_video_path, output_video_name):
    
    cap = cv.VideoCapture(input_video_path)
    
    frame_width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    fps = 60
    total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    codec = cv.VideoWriter_fourcc("M", "P", "4", "V")


    
    out = cv.VideoWriter(output_video_name, codec, fps, (frame_width, frame_height))
    
    count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        


        colorized_frame = color_me2(frame, model)
        colorized_frame = (predict_model(colorized_frame)*255).astype(np.uint8)


        out.write(colorized_frame)

        if count == 30:
            break
        

        count+=1
        print(count, "/", total_frames)

    
    cap.release()
    out.release()
    

In [7]:
now = datetime.now()
name = now.strftime("%m%d%Y_%H%M%S")

input_video = "Test\\VID_48830720_044431_226.mp4"
output_video = f"Test\\{name}.mp4"

colorize_video(input_video, output_video)

NameError: name 'colorize_video' is not defined

In [8]:


now = datetime.now()
name = now.strftime("%m%d%Y_%H%M%S")

input = "Test\\test8.jpeg"
output = f"Test\\{name}.png"

image = cv.imread("Test\\test8.jpeg")
print("shapes: ", image.shape[:])

predicted = (predict_model(image)*255).astype(np.uint8)
# predicted = (predict_model(predicted)*255).astype(np.uint8)


print("shapes: ", predicted.shape[:])
print("dtype: ", predicted.dtype)
print("value:  ", predicted[6,7,2].astype(np.uint8))





shapes:  (259, 194, 3)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
shapes:  (518, 388, 3)
dtype:  uint8
value:   42


In [15]:
# cv.imshow("org", image)


cv.imshow("predicted", predicted)
kernel = np.array([[0, -1, 0], 
                   [-1, 5,-1], 
                   [0, -1, 0]])

gaussian_blur = cv.GaussianBlur(predicted, (3, 3), 5.0)



sharpened = cv.filter2D(gaussian_blur, -1, kernel)


# Sonucu göster
cv.imshow('Unsharp Mask', sharpened)
cv.imshow('gaussian Mask', gaussian_blur)
# cv.imshow('resize', cv.resize(image, dsize=(388, 518)))
cv.waitKey()
cv.destroyAllWindows()

In [ ]:


img = Image.fromarray(predicted)

img.save(output)